<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/cellmagic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cellmagic.ipynb

# メモ

1. このノートブックでcell magicについて勉強する。
2. 元にしたサイト`The cell magics in IPython`はipython notebookの説明のファイルのひとつだが、内容が古いので確認しながら進める。そのためにもColabの環境が最適。
3. このノートブック`cellmagic.ipynb`はColabで開かれ、コードを実行、変更、実験しながら読むことを想定している。

という想定で始めたが、`%load_ext Cython`あたりからついていけなくなったので中断する(20200720)。もう少しPythonの勉強をしてからもどって来よう。

# はじめに

Colabのコードセルではマジック(`magic`)とよばれるコマンド群を実行することができる。

マジックは対話的に、コマンドラインで使ような感じで使える。

マジックにはラインマジックとセルマジックの2種類がある。


* ラインマジックは`%`1つで始まり、行単位で使う。

* セルマジックは2個のパーセント記号`%%`から始まり、ボディ部分を引数として受け取ることができる。セルの最初の行に書く必要があり、通常は効果の範囲もセルの範囲である。

ラインコマンドの`%lsmagic`はすべてのラインコマンドとセルコマンドをリストアップする。


In [ ]:
%lsmagic

In [2]:
%%latex??

このノートブックではセルマジックについて重点的に説明する。

準備として、`matplotlib`と`numpy`を導入して、数値計算とグラフ描画ができるようにしよう。

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 簡単なセルマジック

コードの実行時間測定の`timeit`はラインマジックとしてもセルマジックとしても存在する。

In [ ]:
%timeit np.linalg.eigvals(np.random.rand(100,100))

100 loops, best of 3: 9.03 ms per loop


In [ ]:
%%timeit a = np.random.rand(100, 100)
np.linalg.eigvals(a)

100 loops, best of 3: 8.6 ms per loop


キャプチャー(`%%capture`)マジックはpythonコードの出力(`stdout/err`)をキャプチャーし、不要な場合は捨てる処理を行い、、使う場合は変数に格納したりできる。

In [ ]:
%%capture capt
from __future__ import print_function
import sys
print('Hello stdout')
print('and stderr', file=sys.stderr)

In [ ]:
capt.stdout, capt.stderr

('Hello stdout\n', 'and stderr\n')

In [ ]:
capt.show()

Hello stdout


and stderr


ライトファイル (`%%writefile`) マジックはセルの中身を名前をつけたファイルに書き込むことができる。




In [ ]:
%%writefile foo.py
print('Hello world')

Writing foo.py


In [ ]:
%run foo

Hello world


# Python以外のプログラミング言語を使うためのマジック

セルマジック`%%script`はbash, ruby, perl, zsh, R, などの言語をセルのサブプロセスとして実行できるようにする。

使うには`%%script`の同じ行に使いたいプログラミング言語のパスもしくはシェルコマンドを書く。セルのその後の内容はその言語またはシェルで解釈、実行され、そのサブプロセスの結果はセルによってキャプチャーされセルの出力として表示される。

In [ ]:
%%script python
import sys
print ('hello from Python %s' % sys.version)

hello from Python 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [ ]:
%%script python3
import sys
print('hello from Python: %s' % sys.version)


hello from Python: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


bash, ruby, perl, javascript, などについてはセルコマンドとしてaliasが用意されている。

`%%script <name>`と書いても同じ。

In [ ]:
# %%ruby
# puts "Hello from Ruby #{RUBY_VERSION}"
# エラーになる "Couldn't find program 'ruby'"

In [ ]:
%%script bash
echo "hello from $BASH"

hello from /bin/bash


# 練習問題: インプットされたテキストに行番号をつける

`lnum.py`というファイルを書いて、次に示されるようにする。

=> これはいまのJupyterではできないのではないか？

```
%%script /lnum.py
my first line
my second
more
```
```
0: my first line
1: my second
2: more
---- END ----
```
プログラム自体は%%writefileで書ける。
```
%%writefile /lnum.py
import sys
n=1
for line in sys.stdin:
	print ("%d:	%s" % (n,line))
	n=n+1
```
が、`%%script`で実行できない。`%run`で実行できるが、セルを標準入力にすることができない。入出力について研究すること。

## 出力のキャプチャー

サブプロセスの`stdout/err`への出力をキャプチャーし、Pythonの変数に入れることができる。

In [ ]:
%%bash
echo "hi, stdout"
echo "hello, stderr" >&2

hi, stdout


hello, stderr


In [ ]:
%%bash --out output --err error
echo "hi, stdout"
echo "hello, stderr" >&2

In [ ]:
print(error)
print(output)

hello, stderr

hi, stdout



# バックグラウンド・スクリプト

スクリプトに`--bg`フラグをつけることにより、スクリプトをバックグラウンドで走らせることができる。

バックグラウンドのスクリプトの出力は`--out/err`フラグで変数に格納する。

# いまここ

In [ ]:
%%python --bg --out python_lines
for n in range (10):
    print ("line %d" % n)
end

Starting job # 4 in a separate thread.


バックグラウンド・スレッドの出力を格納すると、それは出力のテキストではなく、｀stdout/err pipes`である。 -- 意味不明。


In [ ]:
python_lines

<_io.BufferedReader name=56>

In [ ]:
print(python_lines.read())

b'line 0\nline 1\nline 2\nline 3\nline 4\nline 5\nline 6\nline 7\nline 8\nline 9\n'


# Cythonマジック関数エクステンション

### Loading the extension

`Cython`マジック・エクステンションには`Cython`コードを扱うためのマジック関数があり、`%load_ext Cython`で使えるようになる。


In [ ]:
%load_ext Cython


The `%%cython_pyximport` magic allows you to enter arbitrary Cython code into a cell. That Cython code is written as a `.pyx` file in the current working directory and then imported using `pyximport`. You have the specify the name of the module that the Code will appear in. All symbols from the module are imported automatically by the magic function.

In \[4\]:


In [ ]:
%%cython_pyximport foo
def f(x):
    return 4.0\*x

/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/foo.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)

Error compiling Cython file:
------------------------------------------------------------
...
def f(x):
    return 4.0\*x             ^
------------------------------------------------------------

foo.pyx:2:14: Unrecognized character


ImportError: ignored

In [ ]:
f(10)

### The %cython magic




Probably the most important magic is the `%cython` magic. This is similar to the `%%cython_pyximport` magic, but doesn't require you to specify a module name. Instead, the `%%cython` magic uses manages everything using temporary files in the `~/.cython/magic` directory. All of the symbols in the Cython module are imported automatically by the magic.

Here is a simple example of a Black-Scholes options pricing algorithm written in Cython:

In \[7\]:


In [ ]:
%%cython
cimport cython
from libc.math cimport exp, sqrt, pow, log, erf

@cython.cdivision(True)
cdef double std_norm_cdf(double x) nogil:
    return 0.5*(1+erf(x/sqrt(2.0)))

@cython.cdivision(True)
def black_scholes(double s, double k, double t, double v,
                 double rf, double div, double cp):
    """Price an option using the Black-Scholes model.
    
    s : initial stock price
    k : strike price
    t : expiration time
    v : volatility
    rf : risk-free rate
    div : dividend
    cp : +1/-1 for call/put
    """
    cdef double d1, d2, optprice
    with nogil:
        d1 = (log(s/k)+(rf-div+0.5*pow(v,2))*t)/(v*sqrt(t))
        d2 = d1 - v*sqrt(t)
        optprice = cp*s*exp(-div*t)*std_norm_cdf(cp*d1) - \
            cp*k*exp(-rf*t)*std_norm_cdf(cp*d2)
    return optprice

UsageError: Cell magic `%%cython` not found.


In [ ]:
black\_scholes(100.0, 100.0, 1.0, 0.3, 0.03, 0.0, \-1)

In [ ]:
%timeit black\_scholes(100.0, 100.0, 1.0, 0.3, 0.03, 0.0, -1)


In \[8\]:



In \[9\]:



1000000 loops, best of 3: 366 ns per loop

Cython allows you to specify additional libraries to be linked with your extension, you can do so with the `-l` flag (also spelled `--lib`). Note that this flag can be passed more than once to specify multiple libraries, such as `-lm -llib2 --lib lib3`. Here's a simple example of how to access the system math library:

In \[10\]:


In [ ]:
%%cython \-lm
from libc.math cimport sin
print 'sin(1)=', sin(1)




You can similarly use the `-I/--include` flag to add include directories to the search path, and `-c/--compile-args` to add extra flags that are passed to Cython via the `extra_compile_args` of the distutils `Extension` class. Please see [the Cython docs on C library usage](http://docs.cython.org/src/tutorial/clibraries.html) for more details on the use of these flags.


### Rmagic Functions Extension


IPython has an `rmagic` extension that contains a some magic functions for working with R via rpy2. This extension can be loaded using the `%load_ext` magic as follows:


In [ ]:
%load_ext rmagic

/usr/local/lib/python3.6/dist-packages/IPython/extensions/rmagic.py:11: UserWarning: The rmagic extension in IPython has moved to `rpy2.ipython`, please see `rpy2` documentation.
  warnings.warn("The rmagic extension in IPython has moved to "



A typical use case one imagines is having some numpy arrays, wanting to compute some statistics of interest on these arrays and return the result back to python. Let's suppose we just want to fit a simple linear model to a scatterplot.

In \[2\]:


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X \= np.array(\[0,1,2,3,4\])
Y \= np.array(\[3,5,4,6,7\])
plt.scatter(X, Y)

SyntaxError: ignored




In \[3\]:



Out\[3\]:

<matplotlib.collections.PathCollection at 0x112ab1f90>


We can accomplish this by first pushing variables to R, fitting a model and returning the results. The line magic %Rpush copies its arguments to variables of the same name in rpy2. The %R line magic evaluates the string in rpy2 and returns the results. In this case, the coefficients of a linear model.

In \[4\]:


In [ ]:
%Rpush X Y
%R lm(Y~X)$coef

It is also possible to return more than one value with %R.

In [ ]:
%R resid(lm(Y~X)); coef(lm(X~Y))






In \[5\]:



One can also easily capture the results of %R into python objects. Like R, the return value of this multiline expression (multiline in the sense that it is separated by ';') is the final value, which is the _coef(lm(X~Y))_. To pull other variables from R, there is one more magic.

There are two more line magics, %Rpull and %Rget. Both are useful after some R code has been executed and there are variables in the rpy2 namespace that one would like to retrieve. The main difference is that one returns the value (%Rget), while the other pulls it to self.shell.user\_ns (%Rpull). Imagine we've stored the results of some calculation in the variable "a" in rpy2's namespace. By using the %R magic, we can obtain these results and store them in b. We can also pull them directly to user\_ns with %Rpull. They are both views on the same data.

In \[6\]:


In [ ]:
b \= %R a=resid(lm(Y~X))
%Rpull a
print(a)
assert id(b.data) \== id(a.data)
%R -o a

# Plotting and capturing output[¶](#Plotting-and-capturing-output)
----------------------------------------------------------------

R's console (i.e. its stdout() connection) is captured by ipython, as are any plots which are published as PNG files like the notebook with arguments --pylab inline. As a call to %R may produce a return value (see above) we must ask what happens to a magic like the one below. The R code specifies that something is published to the notebook. If anything is published to the notebook, that call to %R returns None.

In \[7\]:


In [ ]:
from \_\_future\_\_ import print\_function
v1 \= %R plot(X,Y); print(summary(lm(Y~X))); vv=mean(X)\*mean(Y)
print('v1 is:', v1)
v2 \= %R mean(X)\*mean(Y)
print('v2 is:', v2)




Call:
lm(formula = Y ~ X)

Residuals:
   1    2    3    4    5 
-0.2  0.9 -1.0  0.1  0.2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)   3.2000     0.6164   5.191   0.0139 \*
X             0.9000     0.2517   3.576   0.0374 \*
---
Signif. codes:  0 ‘\*\*\*’ 0.001 ‘\*\*’ 0.01 ‘\*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7958 on 3 degrees of freedom
Multiple R-squared:   0.81,	Adjusted R-squared:  0.7467 
F-statistic: 12.79 on 1 and 3 DF,  p-value: 0.03739


# Cell level magic

Often, we will want to do more than a simple linear regression model. There may be several lines of R code that we want to use before returning to python. This is the cell-level magic.

For the cell level magic, inputs can be passed via the -i or --inputs argument in the line. These variables are copied from the shell namespace to R's namespace using rpy2.robjects.r.assign. It would be nice not to have to copy these into R: rnumpy ( [http://bitbucket.org/njs/rnumpy/wiki/API](https://bitbucket.org/njs/rnumpy/wiki/API) ) has done some work to limit or at least make transparent the number of copies of an array. This seems like a natural thing to try to build on. Arrays can be output from R via the -o or --outputs argument in the line. All other arguments are sent to R's png function, which is the graphics device used to create the plots.

We can redo the above calculations in one ipython cell. We might also want to add some output such as a summary from R or perhaps the standard plotting diagnostics of the lm.

In [ ]:
%%R -i X,Y -o XYcoef
XYlm = lm(Y~X)
XYcoef = coef(XYlm)
print(summary(XYlm))
par(mfrow=c(2,2))
plot(XYlm)

# octavemagic: Octave inside IPython

The octavemagic extension provides the ability to interact with Octave. It depends on the oct2py and h5py packages, which may be installed using easy_install. It has been closely modeled after the R extension, so many of its names and usage patterns are the same.

To enable the extension, load it as follows:


In [ ]:
%load_ext octavemagic

# Overview

Loading the extension enables three magic functions: %octave, %octave_push, and %octave_pull.

The first is for executing one or more lines of Octave, while the latter allow moving variables between the Octave and Python workspace. Here you see an example of how to execute a single line of Octave, and how to transfer the generated value back to Python:

In [ ]:
x = %octave [1 2; 3 4];
x

When using the cell magic, `%%octave` (note the double `%`), multiple lines of Octave can be executed together. Unlike with the single cell magic, no value is returned, so we use the `-i` and `-o` flags to specify input and output variables.

In [ ]:
%%octave -i x -o y
y = x + 3;

In [ ]:
y

# Plotting

Plot output is automatically captured and displayed, and using the `-f` flag you may choose its format (currently, `png` and `svg` are supported).

In [ ]:
%%octave -f svg

p = [12 -2.5 -8 -0.1 8];
x = 0:0.01:1;

polyout(p, 'x')
plot(x, polyval(p, x));

The plot size is adjusted using the `-s` flag:

In [ ]:
%%octave -s 500,500

# butterworth filter, order 2, cutoff pi/2 radians
b = [0.292893218813452  0.585786437626905  0.292893218813452];
a = [1  0  0.171572875253810];
freqz(b, a, 32);

In [ ]:
%%octave -s 600,200 -f png

subplot(121);
[x, y] = meshgrid(0:0.1:3);
r = sin(x - 0.5).^2 + cos(y - 0.5).^2;
surf(x, y, r);

subplot(122);
sombrero()

# 参考サイト

1. [The cell magics in IPython](https://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb)
